# Poizvedovanje s SPARQL v okolju Python

V tem delovnem zvezku prikazujemo uporabo storitve SPARQL, ki jo lahko nudijo semantične podatkovne baze. Kot primer uporabimo storitev DBPedije.

Primer bomo prikazali z uporabi knjižnic [SPARQLWrapper](https://rdflib.github.io/sparqlwrapper/) in [RDFLib](https://rdflib.readthedocs.io/en/stable/).


In [1]:
#Namestitev knjižnic
!pip install SPARQLWrapper==2.0.0 rdflib==7.5.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.2/587.2 kB 7.3 MB/s eta 0:00:00


## SPARQLWrapper

SPARQLWrapper je enostavna in 'lahka' Python programska knjižnica, ki omogoča komunikacijo s storitvijo SPARQL. V ozadju se uporablja tudi s strani knjižnice RDFLib.

V spodnjem primeru prikazujemo izvedbo poizvedb SPARQL z javno storitvijo [DBPedia SPARQL Endpoint](http://dbpedia.org/sparql).

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Pridobi angleško ime države (npr. Slovenia)
#country = str(input("Enter country name: "))
country = "Slovenia"

# Pripravi poizvedbo
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX dbp:	<http://dbpedia.org/property/>
SELECT ?country ?area ?currency ?longName ?pmName ?presidentName
WHERE {
    ?country rdf:type dbo:Country .
    ?country dbo:area ?area .
    ?country dbp:conventionalLongName ?longName .
    ?country dbo:currency ?currency .
    ?country dbp:leaderName ?primeMinister .
    ?primeMinister dbp:office dbr:Prime_Minister_of_Slovenia .
    ?primeMinister dbp:name ?pmName .
    ?country dbp:leaderName ?president .
    ?president dbp:office dbr:President_of_Slovenia .
    ?president dbp:name ?presidentName .
    FILTER regex(?longName, "{}", "i")
}
"""
query = query.replace("{}", country)

# Poveži se na storitev 'DBPedia SPARQL endpoint' in izvedi poizvedbo
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Izpiši prvi zadetek
print("Country data:")
result = results["results"]["bindings"][0]
print(f"\t{result['longName']['value']}\n\t\tArea [km2]: {float(result['area']['value'])/1000000:.2f}\n\t\t" + \
      f"Currency: {result['currency']['value']}\n\t\tPrime minister: '{result['pmName']['value']}'\n\t\tPresident: '{result['presidentName']['value']}'")

Country data:
	Republic of Slovenia
		Area [km2]: 20271.00
		Currency: http://dbpedia.org/resource/Euro
		Prime minister: 'Robert Golob'
		President: 'Nataša Pirc Musar'


## RDFLib

Knjižnica RDFLib ponuja prijaznejši vmesnik za neposredno delo z ontologijo in s storitvami SPARQL.

V spodnjem primeru prikazujemo dva primera uporabe z javno storitvijo DBPedia SPARQL Endpoint.

In [ ]:
from rdflib.plugins.stores.sparqlstore import SPARQLStore, SPARQLUpdateStore
from rdflib import URIRef, Namespace, Graph

# Zahtevaj vrnjene rezultate v formatu JSON (morebitne težave pri razčljenjevanju XML/RDF)
store = SPARQLStore(query_endpoint="https://dbpedia.org/sparql", returnFormat="json")
store.method = "GET"

graph = Graph(store=store)

# Definicija imenskih prostorov
dbo = Namespace("http://dbpedia.org/ontology/")
dbr = Namespace("http://dbpedia.org/resource/")

# Primer A
population = graph.value(dbr.Ljubljana, dbo.populationTotal)
print(f"Glede na podatke na DBPediji, v Ljubljani živi {population} prebivalcev.")

# Primer B
population = graph.value(URIRef("http://dbpedia.org/resource/Ljubljana"), dbo.populationTotal)
print(f"Glede na podatke na DBPediji, v Ljubljani živi {population} prebivalcev.")

# Primer C
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
SELECT ?population WHERE {
  dbr:Ljubljana dbo:populationTotal ?population .
}
"""
print(f"Glede na podatke na DBPediji, v Ljubljani živi {list(graph.query(query))[0].population} prebivalcev.")

Glede na podatke na DBPediji, v Ljubljani živi 300354 prebivalcev.
Glede na podatke na DBPediji, v Ljubljani živi 300354 prebivalcev.
Glede na podatke na DBPediji, v Ljubljani živi 300354 prebivalcev.
